### Library Imports

In [1]:
import numpy
import torch
import transformers
import json
import ast

/dlabscratch1/berquet/conda/envs/env2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
print("Python version")
print(sys.version)
print("Version info.")
print(sys.version_info)

Python version
3.10.14 (main, Mar 21 2024, 16:24:04) [GCC 11.2.0]
Version info.
sys.version_info(major=3, minor=10, micro=14, releaselevel='final', serial=0)


In [3]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

### Load Dataset

In [4]:
dataset = load_dataset("berquetR/dlab_project_optimal_links")

In [ ]:
test_dataset = dataset['test']

In [ ]:
test_dataset

In [ ]:
test_no_prompt = dataset['test']

In [ ]:
train_dataset = dataset['train']

In [ ]:
train_no_prompt = dataset['train']

In [7]:
import lmql
import nest_asyncio

nest_asyncio.apply()

chain_of_thought = lmql.query("""
    "Q: {question}\\n"
    "A: Let's think step by step.\\n"
    "[ANSWER]"
""", is_async=False)

chain_of_thought("What is 2x3?")


/dlabscratch1/berquet/conda/envs/env2/lib/python3.10/site-packages/lmql/runtime/bopenai/batched_openai.py:507: OpenAIAPIWarning: OpenAI API: Underlying stream of OpenAI complete() call failed with error

To use openai/<models>, you have to configure your API credentials. To do so
you can either define the `OPENAI_API_KEY` environment variable or create a
file `api.env` in one of the following locations:

 - /dlabscratch1/berquet/conda/envs/env2/lib/python3.10/site-packages/api.env
 - /dlabscratch1/berquet/project/dlab_project/api.env
 - /dlabscratch1/berquet/home/.lmql/api.env

To use OpenAI models you need to create an api.env file with the following
contents:
        
        openai-secret: <your openai secret>
        openai-org: <your openai org>
        
For more info, check the related project docs:
https://docs.lmql.ai/en/stable/language/openai.html#configuring-openai-api-credentials
 (<class 'FileNotFoundError'>)

Retrying... (attempt: 0)
  warnings.warn(f"OpenAI API: Underlyin

/dlabscratch1/berquet/conda/envs/env2/lib/python3.10/site-packages/lmql/runtime/bopenai/batched_openai.py:507: OpenAIAPIWarning: OpenAI API: Underlying stream of OpenAI complete() call failed with error

To use openai/<models>, you have to configure your API credentials. To do so
you can either define the `OPENAI_API_KEY` environment variable or create a
file `api.env` in one of the following locations:

 - /dlabscratch1/berquet/conda/envs/env2/lib/python3.10/site-packages/api.env
 - /dlabscratch1/berquet/project/dlab_project/api.env
 - /dlabscratch1/berquet/home/.lmql/api.env

To use OpenAI models you need to create an api.env file with the following
contents:
        
        openai-secret: <your openai secret>
        openai-org: <your openai org>
        
For more info, check the related project docs:
https://docs.lmql.ai/en/stable/language/openai.html#configuring-openai-api-credentials
 (<class 'FileNotFoundError'>)

Retrying... (attempt: 8)
  warnings.warn(f"OpenAI API: Underlyin

/dlabscratch1/berquet/conda/envs/env2/lib/python3.10/site-packages/lmql/runtime/bopenai/batched_openai.py:507: OpenAIAPIWarning: OpenAI API: Underlying stream of OpenAI complete() call failed with error

To use openai/<models>, you have to configure your API credentials. To do so
you can either define the `OPENAI_API_KEY` environment variable or create a
file `api.env` in one of the following locations:

 - /dlabscratch1/berquet/conda/envs/env2/lib/python3.10/site-packages/api.env
 - /dlabscratch1/berquet/project/dlab_project/api.env
 - /dlabscratch1/berquet/home/.lmql/api.env

To use OpenAI models you need to create an api.env file with the following
contents:
        
        openai-secret: <your openai secret>
        openai-org: <your openai org>
        
For more info, check the related project docs:
https://docs.lmql.ai/en/stable/language/openai.html#configuring-openai-api-credentials
 (<class 'FileNotFoundError'>)

Retrying... (attempt: 18)
  warnings.warn(f"OpenAI API: Underlyi

MaximumRetriesExceeded: Maximum retries exceeded (21) with error <class 'FileNotFoundError'>: To use openai/<models>, you have to configure your API credentials. To do so
you can either define the `OPENAI_API_KEY` environment variable or create a
file `api.env` in one of the following locations:

 - /dlabscratch1/berquet/conda/envs/env2/lib/python3.10/site-packages/api.env
 - /dlabscratch1/berquet/project/dlab_project/api.env
 - /dlabscratch1/berquet/home/.lmql/api.env

To use OpenAI models you need to create an api.env file with the following
contents:
        
        openai-secret: <your openai secret>
        openai-org: <your openai org>
        
For more info, check the related project docs:
https://docs.lmql.ai/en/stable/language/openai.html#configuring-openai-api-credentials


### Build Prompt

In [9]:
# Function to format a row according to your fine-tuning requirements
def format_row(row):
    input_data = {
        "Source": row['current_page'], 
        "Candidates": row['current_page_links'], 
        "Target": row['target']
    }
    
    prompt = f"""You are a knowledge discovery expert familiar with the Wikipedia link structure and your objective is to play the game of Wikispeedia: https://dlab.epfl.ch/wikispeedia/play/.
##Goal 
Given two Wikipedia articles, a source and a target, your goal is to reach the target article starting from the source article in as few clicks as possible. For the articles you are given this is always possible.

##Constraint 
You should exclusively follow the links present in the articles that you encounter along the way.

##Fine-grained instructions 
1. While the overall goal is to find a path from a source to a target article, you will proceed step by step.
2. Given outgoing links from the source article as candidates, you should select the candidate that takes you closer to the target article. Use your knowledge of the "expected" Wikipedia link structure and relatedness between articles to identify the candidate that takes you closer to the target.
3. Choose **only** from the provided candidates.
4. Do not provide an algorithm, code to solve the task, or explanation just provide the link the choose among candidates.
6. Even though the proposed links are not related to the target you should **always** choose a link.

##Input 
{json.dumps(input_data, indent=4)}


"""
    return prompt

In [10]:
# Apply the formatting function to each row
test_dataset = test_dataset.map(lambda x: {"text": format_row(x)})

# You might want to remove the old columns and keep only 'text'
test_dataset = test_dataset.remove_columns(['source', 'target', 'current_page', 'current_page_links', 'next_page', '__index_level_0__'])

In [11]:
train_dataset = train_dataset.map(lambda x: {"text": format_row(x)})

# You might want to remove the old columns and keep only 'text'
train_dataset = train_dataset.remove_columns(['source', 'target', 'current_page', 'current_page_links', 'next_page', '__index_level_0__'])

### Import model

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
model_id = "microsoft/phi-1_5"
model = AutoModelForCausalLM.from_pretrained("phi15_run1_optipaths/checkpoint-9000", quantization_config=bnb_config, device_map="auto")

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast = False)

In [16]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(model, "phi15_run1_optipaths/checkpoint-9000")

In [17]:
model_base = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

### Output extraction 

In [18]:
def extract_output_from_prompt(prompt):
    # Look for the start of the JSON output block
    start = prompt.find('##Output')
    if start == -1:
        return "Output block not found."

    # Extract the part of the prompt after the "##Output" marker
    output_part = prompt[start:]

    # Find the actual output value inside curly braces
    output_start = output_part.find('{')
    output_end = output_part.find('}')
    if output_start == -1 or output_end == -1:
        return "Output not formatted correctly."

    # Extract the string inside the curly braces
    output_content = output_part[output_start+1:output_end]

    # Look for the value after "Output":
    marker = "Output\": "
    value_start = output_content.find(marker)
    if value_start == -1:
        return "Output key not found."

    # Extract and clean the output value
    output_value = output_content[value_start + len(marker):].strip("\" ")
    
    formatted_output = output_value.replace('_', ' ')
    
    return formatted_output[:-2] 

In [19]:
def is_word_in_links(word, current_page_links_str):
    # Handle escaped single quotes
    if "\\'" in current_page_links_str:
        current_page_links_str = current_page_links_str.replace("\\'", "'") 
    
    # Convert string representation of list to actual list
    links_list = ast.literal_eval(current_page_links_str)

    # Check if the word is in the list
    return word in links_list

### Random tests finetuned model

In [20]:
index = 30

In [21]:
input = test_dataset[index]
input_tok = tokenizer([input['text']],return_tensors = "pt")

In [24]:
ft_model.eval()
with torch.no_grad():
    out = (tokenizer.decode(ft_model.generate(**input_tok, max_new_tokens=150)[0], skip_special_tokens=True))
    out_extracted = extract_output_from_prompt(out)
    print(out_extracted)

Arctic Ocean


In [25]:
out

'You are a knowledge discovery expert familiar with the Wikipedia link structure and your objective is to play the game of Wikispeedia: https://dlab.epfl.ch/wikispeedia/play/.\n##Goal \nGiven two Wikipedia articles, a source and a target, your goal is to reach the target article starting from the source article in as few clicks as possible. For the articles you are given this is always possible.\n\n##Constraint \nYou should exclusively follow the links present in the articles that you encounter along the way.\n\n##Fine-grained instructions \n1. While the overall goal is to find a path from a source to a target article, you will proceed step by step.\n2. Given outgoing links from the source article as candidates, you should select the candidate that takes you closer to the target article. Use your knowledge of the "expected" Wikipedia link structure and relatedness between articles to identify the candidate that takes you closer to the target.\n3. Choose **only** from the provided candi

In [ ]:
test_no_prompt[index]['current_page_links']

In [ ]:
is_word_in_links(out_extracted , test_no_prompt[index]['current_page_links'])

In [ ]:
test_no_prompt[index]['next_page']

### Tests : chosen link is among the proposed ones and chosen link is same as human

In [180]:
matching = 0
non_matching = {}

In [181]:
good_prediction = 0 
bad_prediction = {}

In [182]:
for i in range(100) :
    input = test_dataset[i + 232]
    input_tok = tokenizer([input['text']],return_tensors = "pt").to("cuda")
    with torch.no_grad():
        out = (tokenizer.decode(ft_model.generate(**input_tok, max_new_tokens=25)[0], skip_special_tokens=True))
        out_extracted = extract_output_from_prompt(out)
    links =  test_no_prompt[i]['current_page_links']
    
    #increase counter to get a sense of the proportion of wrong chosing
    if (is_word_in_links(out_extracted ,links)) : 
        matching += 1    
        true_target = test_no_prompt[i]['next_page'].replace('_', ' ')
        if true_target == out_extracted :
            good_prediction += 1
        else : 
            print(f'Bad prediction on iter :{i}')
            if not (true_target in bad_prediction) : 
                bad_prediction[true_target] = []
            bad_prediction[true_target].append(out_extracted)
            
    else : 
        ## Keep track of the non matching chosen links and list of links to look for some patterns
        if not(out_extracted in non_matching) :
            non_matching[out_extracted] = []
        print(f'Chosen link not in proposed on iter{i}')
        non_matching[out_extracted].append(links)


Chosen link not in proposed on iter0
Chosen link not in proposed on iter1
Chosen link not in proposed on iter2
Bad prediction on iter :3
Chosen link not in proposed on iter4
Chosen link not in proposed on iter5
Bad prediction on iter :6
Chosen link not in proposed on iter7
Chosen link not in proposed on iter8
Chosen link not in proposed on iter9
Chosen link not in proposed on iter10
Chosen link not in proposed on iter11
Bad prediction on iter :12
Chosen link not in proposed on iter13
Chosen link not in proposed on iter14
Chosen link not in proposed on iter15


KeyboardInterrupt: 

In [ ]:
good_prediction

In [184]:
non_matching

{'Coconut oil': ["['History of the Internet', 'Clay', 'Abacus', 'Computers', 'Abacus', 'China', 'Abacus', 'Abacus', 'Johannes Kepler', 'Blaise Pascal', 'Gottfried Leibniz', 'Leibniz', 'Charles Babbage', 'Moon', 'Charles Babbage', 'United States Constitution', 'World War II', 'Computer programming', 'Charles Babbage', 'Richard Feynman', 'Integrated circuit', 'World War II', 'Computer programming', 'World War II', 'Electronics', 'Mercury (element)', 'Alan Turing', 'Alan Turing', 'Computer science', 'Algorithm', 'Germany', 'Charles Babbage', 'Programming language', 'World War II', 'World War II', 'World War II', 'Alan Turing', 'Boolean logic', 'John von Neumann', 'Electronics', 'University of Cambridge', 'Soviet Union', 'Ukraine', 'Programming language', 'Central processing unit', 'Integrated circuit']"],
 'List of essential oils': ['[\'Battle of Normandy\', \'Nuclear weapon\', \'United States\', \'Soviet Union\', \'Europe\', \'Cold War\', \'United Kingdom\', \'United States\', \'Nazi Ger

In [ ]:
bad_prediction

### Tests : random choosing among proposed links

### Tests : simulating an agent

In [55]:
page_name_links = load_dataset("berquetR/file_names_links")

In [61]:
def format_agent_prompt(source, target, source_links):
    input_data = {
        "Source": source, 
        "Candidates": source_links,
        "Target": target
    }
    
    prompt = f"""You are a knowledge discovery expert familiar with the Wikipedia link structure and your objective is to play the game of Wikispeedia: https://dlab.epfl.ch/wikispeedia/play/.
##Goal 
Given two Wikipedia articles, a source and a target, your goal is to reach the target article starting from the source article in as few clicks as possible. For the articles you are given this is always possible.

##Constraint 
You should exclusively follow the links present in the articles that you encounter along the way.

##Fine-grained instructions 
1. While the overall goal is to find a path from a source to a target article, you will proceed step by step.
2. Given outgoing links from the source article as candidates, you should select the candidate that takes you closer to the target article. Use your knowledge of the "expected" Wikipedia link structure and relatedness between articles to identify the candidate that takes you closer to the target.
3. Choose **only** from the provided candidates.
4. Do not provide an algorithm or code to solve the task, instead give only the solution.

##Input 
{json.dumps(input_data, indent=4)}


"""
    return prompt

In [67]:
def links_from_source(dataset, source):
    filtered = dataset['train'].filter(lambda example : example['filename'] == source)

    if len(filtered) > 0:
        row_with_filename_e = filtered[0]
        links = row_with_filename_e['links']
        return links
    else:
        print("No rows found where filename = 'e'")

In [161]:
test_var = test_no_prompt[1013]
test_var

{'source': 'Concorde',
 'target': 'Periodic_table_(large_version)',
 'current_page': 'Concorde',
 'current_page_links': "['September 11, 2001 attacks', 'United Kingdom', 'France', 'United States', 'Soviet Union', 'New York City', 'Harold Macmillan', 'Scotland', 'French language', 'Bristol', '1973 oil crisis', 'France', 'John F. Kennedy', 'Airbus', 'BAE Systems', 'Airbus', 'Boeing 747', 'Mexico', 'Bristol', 'London', 'Bahrain', 'Paris', 'Rio de Janeiro', 'Atlantic Ocean', 'New York City', 'Washington, D.C.', 'Supreme Court of the United States', 'Barbados', 'Finland', 'Malaysia', 'India', 'Mexico City', 'Boeing 747', 'France', 'Titanium', 'September 11, 2001 attacks', 'Barbados', 'Boston, Massachusetts', 'United Kingdom', 'Birmingham', 'Belfast', 'Manchester', 'Cardiff', 'Edinburgh', 'Elizabeth II of the United Kingdom', 'Windsor Castle', 'Airbus', 'Germany', 'Airbus', 'TGV', 'BBC', 'Mini', 'Supermarine Spitfire', 'Airbus', 'Doctor Who']",
 'next_page': 'Titanium',
 '__index_level_0__':

In [162]:
source = test_var['current_page']
target = test_var['target']
links = links_from_source(page_name_links , source)
next_page = source
iter_i = 0 

Filter: 100%|█████████████████████| 5409/5409 [00:00<00:00, 42056.40 examples/s]


In [163]:
while source != target : 
    
    print(f'Iteration : {iter_i}')
    print(f'Source : {source}')
    print(f'Target : {target}')
    
    #Prompt Model
    prompt = format_agent_prompt(source, target, links)
    prompt_tokenized = tokenizer([prompt],return_tensors = "pt").to("cuda")
    
    #Extract output
    out = (tokenizer.decode(ft_model.generate(**prompt_tokenized, max_new_tokens=30)[0], skip_special_tokens=True))
    out_extracted = extract_output_from_prompt(out)
    print(f'Model output : {out_extracted}')
    print(f'Links : {links}')
    
    if (is_word_in_links(out_extracted ,links)) : 
        #If model chose a next_page among the one proposed
        print(f'Model chose next_page among links, Next Page : {out_extracted}')
        source = out_extracted.replace(' ','_')
        links = links_from_source(page_name_links , source)
        iter_i += 1 
    else : 
        print(f'Model did not chose next_page among links')
        break
        

Iteration : 0
Source : Concorde
Target : Periodic_table_(large_version)
Model output : Boeing 747
Links : ['September 11, 2001 attacks', 'United Kingdom', 'France', 'United States', 'Soviet Union', 'New York City', 'Harold Macmillan', 'Scotland', 'French language', 'Bristol', '1973 oil crisis', 'France', 'John F. Kennedy', 'Airbus', 'BAE Systems', 'Airbus', 'Boeing 747', 'Mexico', 'Bristol', 'London', 'Bahrain', 'Paris', 'Rio de Janeiro', 'Atlantic Ocean', 'New York City', 'Washington, D.C.', 'Supreme Court of the United States', 'Barbados', 'Finland', 'Malaysia', 'India', 'Mexico City', 'Boeing 747', 'France', 'Titanium', 'September 11, 2001 attacks', 'Barbados', 'Boston, Massachusetts', 'United Kingdom', 'Birmingham', 'Belfast', 'Manchester', 'Cardiff', 'Edinburgh', 'Elizabeth II of the United Kingdom', 'Windsor Castle', 'Airbus', 'Germany', 'Airbus', 'TGV', 'BBC', 'Mini', 'Supermarine Spitfire', 'Airbus', 'Doctor Who']
Model chose next_page among links, Next Page : Boeing 747


Filter: 100%|█████████████████████| 5409/5409 [00:00<00:00, 76305.22 examples/s]

Iteration : 1
Source : Boeing_747
Target : Periodic_table_(large_version)


Model output : Airbus A380
Links : ['Airbus A380', 'Soviet Union', 'Hong Kong', 'London', 'Sydney', 'Seattle, Washington', 'Concorde', 'Airbus A380', 'Airbus', 'Airbus A380', 'Piano', 'Japan', 'Airbus', 'Airbus A380', 'Airbus A380', 'Japan', 'President of the United States', 'President of the United States', 'Bahrain', 'Brunei', 'Iran', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates', 'Seattle, Washington', 'United States', 'Netherlands', 'Australia', 'Johannesburg', 'South Africa', 'Germany', 'Paris', 'France', 'Tehran', 'Iran', 'South Korea', 'Nairobi', 'Pan Am Flight 103', 'September 11, 2001', 'Hong Kong', 'Taipei', 'Taiwan', 'Taipei', 'Addis Ababa', 'James Bond', 'Airbus A380', 'Airbus A380']
Model chose next_page among links, Next Page : Airbus A380
Iteration : 2
Source : Airbus_A380
Target : Periodic_table_(large_version)
Model output : Boeing 747
Links : ['Airbus', 'France', 'Boeing 747', 'Chicago', 'Sydney', 'Boeing 747', 'Boeing 747', 'Euro', 'Singapore', 'Brisbane', 

KeyboardInterrupt: 

In [ ]:
train_no_prompt[200]